In [1]:
import pandas as pd
import re

ImportError: No module named 'pandas'

In [2]:
import os
from os.path import expanduser

In [5]:
ECOINVENT_VERSION = '3.2'

datapath = os.path.join(expanduser('~'), 'Dropbox', 'data', 'Ecoinvent', ECOINVENT_VERSION)
wbfile = 'activity_overview_for_users_' + ECOINVENT_VERSION + '_undefined.xlsx'
sheetname = 'activity overview'

In [6]:
A = pd.read_excel(os.path.join(datapath,wbfile),sheetname)

In [7]:
A.columns

Index([u'activity uuid', u'activityName', u'geography', u'start date',
       u'end date', u'ISIC number', u'ISIC class', u'specialActivityType',
       u'inheritence status', u'tags', u'technologyLevel',
       u'constrained market', u'group', u'product name', u'unit',
       u'APOS classification', u'cut-off classification'],
      dtype='object')

In [18]:
activities = A['activityName'].unique()

In [19]:
len(activities)

6046

In [23]:
activities[0]

u'treatment of used laptop computer, manual dismantling'

In [28]:
atags = tags_from_list(activities,'\s*[,;]\s*')
len(atags)

5102

In [5]:
help(re.split)

Help on function split in module re:

split(pattern, string, maxsplit=0, flags=0)
    Split the source string by the occurrences of the pattern,
    returning a list containing the resulting substrings.



In [26]:
def tags_from_list(items, delimiter=','):
    tags = set()
    for v in items:
        tags = tags.union(re.split(delimiter, v))
    return tags
def tags_from_frame(frame, column, delimiter=','):
    return tags_from_list(frame[column].unique(), delimiter)


In [29]:
all_tags = list(tags_from_frame(A, 'activityName', delimiter='\s*[,:]\s*'))

In [102]:
A['ISIC class'].unique()[:10]

array([u'Materials recovery', u'Manufacture of basic chemicals',
       u'Manufacture of plastics and synthetic rubber in primary forms',
       u'Logging', u'Manufacture of power-driven hand tools',
       u'Treatment and disposal of non-hazardous waste',
       u'Electric power generation, transmission and distribution',
       u'Sewerage', u'Sawmilling and planing of wood',
       u'Treatment and coating of metals; machining'], dtype=object)

In [42]:
tx = A[A['activityName'].str.contains('transport')]
len(tx)

801

In [8]:
for i,v in lorry[:10].iterrows():
    print(" %s -- %s -- %s" % (v['activityName'], v['geography'], v['product name']))

 lorry production, 16 metric ton -- GLO -- used lorry, 16 metric ton
 lorry production, 16 metric ton -- GLO -- wastewater from lorry production
 lorry production, 16 metric ton -- GLO -- wastewater, average
 lorry production, 16 metric ton -- GLO -- lorry, 16 metric ton
 lorry production, 16 metric ton -- RER -- wastewater from lorry production
 lorry production, 16 metric ton -- RER -- wastewater, average
 lorry production, 16 metric ton -- RER -- used lorry, 16 metric ton
 lorry production, 16 metric ton -- RER -- lorry, 16 metric ton
 lorry production, 28 metric ton -- GLO -- used lorry, 28 metric ton
 lorry production, 28 metric ton -- GLO -- wastewater from lorry production


In [9]:
tags = list(tags_from_frame(lorry, 'activityName', delimiter='\s*[,:]\s*'))

In [10]:
tags[12]

u'treatment of used lorry'

In [103]:
A[A['activityName'].str.contains(tags[12])][:10]

KeyError: 12

In [149]:
def count_tags_in_frame(df, column='activityName', delimiter='\s*[,;]\s*'):
    all_tags = tags_from_frame(df, column, delimiter)
    tag_count = {}
    for tag in all_tags:
        tag_count[tag] = df[column].str.contains(tag).sum()
    tag_count_frame = pd.DataFrame.from_dict(tag_count,'index')
    tag_count_frame.columns = ['count']
    c = tag_count_frame.sort_values('count', ascending=False)
    return c, tag_count
    #return c[c['count'] > 0]
    

In [44]:
tag_count_frame = pd.DataFrame.from_dict(tag_count, 'index'

In [46]:
tag_count_frame.columns = ['count']

In [47]:
c = tag_count_frame.sort_values('count', ascending=False)

In [150]:
c, tc = count_tags_in_frame(tx)

c:\python27\lib\site-packages\ipykernel\__main__.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [151]:
tc

{u'3.5-7.5 ton': 80,
 u'7.5-16 ton': 80,
 u'CO2 refrigerant': 8,
 u'EURO 3': 85,
 u'EURO 4': 85,
 u'EURO 5': 90,
 u'EURO3': 90,
 u'EURO3 to generic market for transport': 2,
 u'EURO4': 90,
 u'EURO4 to generic market for transport': 2,
 u'EURO5': 90,
 u'EURO5 to generic market for transport': 2,
 u'EURO6': 82,
 u'EURO6 to generic market for transport': 2,
 u'LTO cycle': 2,
 u'R134a refrigerant': 88,
 u'agricultural': 3,
 u'aircraft': 14,
 u'aircraft with reefer': 4,
 u'all sizes': 8,
 u'barge': 14,
 u'barge tanker': 5,
 u'barge with reefer': 4,
 u'bicycle': 17,
 u'carbon dioxide': 80,
 u'cooling': 91,
 u'cooling to generic market': 8,
 u'diesel': 87,
 u'electric': 28,
 u'electric bicycle': 14,
 u'electric scooter': 5,
 u'electricity': 11,
 u'freezing': 91,
 u'freezing to generic market': 8,
 u'freight': 432,
 u'freight train': 26,
 u'helicopter': 4,
 u'high-speed': 4,
 u'inland waterways': 14,
 u'intercontinental': 4,
 u'intracontinental': 4,
 u'label-certified electricity': 7,
 u'large

In [68]:
tags = {}
for i,v in enumerate(c.index): tags[v] = i

In [98]:
tags['1']

5

In [116]:
tags[:10]

TypeError: unhashable type

In [16]:
%timeit -n100 A['activityName'].str.contains(tags[12]).sum()

100 loops, best of 3: 20.3 ms per loop


In [17]:
%timeit -n100 len(A[A['activityName'].str.contains(tags[12])])

100 loops, best of 3: 20.1 ms per loop


In [52]:
import numpy

In [55]:
common = numpy.ndarray((len(c), len(c)))

In [60]:
c.__dict__

{'_data': BlockManager
 Items: Index([u'count'], dtype='object')
 Axis 1: Index([u'transport', u'freight', u'lorry', u'passenger', u'passenger car',
        u'1', u'with refrigeration machine', u'refrigeration machine',
        u'lorry with refrigeration machine', u'market for transport',
        ...
        u'EURO3 to generic market for transport',
        u'EURO6 to generic market for transport',
        u'EURO5 to generic market for transport', u'with particle filter',
        u'regional', u'urban', u'LTO cycle', u'long-distance', u'petol',
        u'market group for transport'],
       dtype='object', length=110)
 IntBlock: slice(0, 1, 1), 1 x 110, dtype: int64,
 '_iloc': <pandas.core.indexing._iLocIndexer at 0x77c70b0>,
 '_item_cache': {},
 'is_copy': <weakref at 07664750; to 'DataFrame' at 07CC1DF0>}

In [70]:
mytags = re.split('\s*[,;]\s*', tx.iloc[0]['activityName'])

In [71]:
mytags

[u'market for transport', u'freight', u'aircraft']

In [76]:
inds = [tags[k] for k in mytags]

In [77]:
inds

[9, 1, 51]

In [78]:
from itertools import combinations

In [83]:
pairs = [k for k in combinations(inds,2)]

In [91]:
common = numpy.zeros(shape=(len(c),len(c)))

0.0

In [94]:
for pair in pairs:
    common[pair[0], pair[1]] +=1
    common[pair[1], pair[0]] +=1

In [96]:
common[9,:]

array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [128]:
c = count_tags_in_frame(tx)

tag_index = {}
for i, v in enumerate(c.index): tag_index[v] = i

common = numpy.zeros(shape=(len(c), len(c)))

for q,record in tx.iterrows():
    mytags = re.split('\s*[,;]\s*', record['activityName'])
    try:
        inds = [tag_index[k] for k in mytags]  # indices into tag list
    except KeyError:
        print 'No key match for %s' % mytags
    pairs = [k for k in combinations(inds, 2)]
    for pair in pairs:
        common[pair[0], pair[1]] += 1
        common[pair[1], pair[0]] += 1


c:\python27\lib\site-packages\ipykernel\__main__.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


No key match for [u'market for transport', u'freight', u'lorry with refrigeration machine', u'7.5-16 ton', u'EURO4', u'carbon dioxide', u'liquid refri(...)_12']
No key match for [u'market for transport', u'freight', u'lorry with refrigeration machine', u'7.5-16 ton', u'EURO5', u'carbon dioxide', u'liquid refri(...)_16']
No key match for [u'market for transport', u'freight', u'lorry with refrigeration machine', u'7.5-16 ton', u'EURO4', u'carbon dioxide', u'liquid refrig(...)_6']
No key match for [u'market for transport', u'freight', u'lorry with refrigeration machine', u'3.5-7.5 ton', u'EURO3', u'carbon dioxide', u'liquid refr(...)_11']
No key match for [u'market for transport', u'freight', u'lorry with refrigeration machine', u'3.5-7.5 ton', u'EURO3', u'carbon dioxide', u'liquid refri(...)_1']
No key match for [u'market for transport', u'freight', u'lorry with refrigeration machine', u'3.5-7.5 ton', u'EURO6', u'carbon dioxide', u'liquid refr(...)_13']
No key match for [u'market for tra

In [133]:
u = tx[tx['activityName'].str.contains('_12$')]['activityName'].iloc[0]

In [134]:
u in tx['activityName'].unique()

True

In [137]:
for i in tags_from_list([u],'\s*[,;]\s*'):
    tag_index[i]

KeyError: u'liquid refri(...)_12'

In [120]:
tx.iloc[0]['activityName']

u'market for transport, freight, aircraft'

In [140]:
c = count_tags_in_frame(tx, delimiter='\s*[,;]\s*')

c:\python27\lib\site-packages\ipykernel\__main__.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [144]:
c[c.index.str.contains('t$')]

,count
transport,801
freight,432
market for transport,140
R134a refrigerant,88
liquid refrigerant,64
aircraft,14
CO2 refrigerant,8
cooling to generic market,8
freezing to generic market,8
EURO3 to generic market for transport,2


In [53]:
help(numpy.ndarray)

Help on class ndarray in module numpy:

class ndarray(__builtin__.object)
 |  ndarray(shape, dtype=float, buffer=None, offset=0,
 |          strides=None, order=None)
 |  
 |  An array object represents a multidimensional, homogeneous array
 |  of fixed-size items.  An associated data-type object describes the
 |  format of each element in the array (its byte-order, how many bytes it
 |  occupies in memory, whether it is an integer, a floating point number,
 |  or something else, etc.)
 |  
 |  Arrays should be constructed using `array`, `zeros` or `empty` (refer
 |  to the See Also section below).  The parameters given here refer to
 |  a low-level method (`ndarray(...)`) for instantiating an array.
 |  
 |  For more information, refer to the `numpy` module and examine the
 |  the methods and attributes of an array.
 |  
 |  Parameters
 |  ----------
 |  (for the __new__ method; see Notes below)
 |  
 |  shape : tuple of ints
 |      Shape of created array.
 |  dtype : data-type, opti